# Understanding Spark DataFrames and Spark SQL

## Learning Objective

* [Creating DataFrames and performing DataFrame Operations (aka Untyped Dataset Operations)](#Creating-DataFrames-and-performing-DataFrame-Operations)
* [Converting DataFrames to RDDs](#Converting-DataFrames-to-RDDs)
* [Running SQL Queries Programmatically](#Running-SQL-Queries-Programmatically)
* [Using SQL functions to perform calculations on data (e.g. built-in Scalar and Aggregate functions)](#Using-SQL-functions-to-perform-calculations-on-data)
* [Table catalog and Hive metastore](#Table-catalog-and-Hive-metastore)
* [Global Temporary View](#Global-Temporary-View)
* [Saving and loading DataFrame data](#Saving-and-loading-DataFrame-data)

At the end, see [Summary](#Summary).

## Starting Point: SparkSession

SparkSession in Spark 2.0 provides builtin support for Hive features including the ability to write queries using HiveQL, access to Hive UDFs, and the ability to read data from Hive tables. To use these features, you do not need to have an existing Hive setup.

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Understanding Spark DataFrames and Spark SQL") \
    .getOrCreate()

## Creating DataFrames and performing DataFrame Operations

(aka Untyped Dataset Operations)

With a SparkSession, applications can create DataFrames from an existing RDD, from a Hive table, or from Spark data sources.

DataFrame’s DSL has a set of functionalities similar to the usual SQL functions for manipulating data in relational databases. DataFrames work like RDDs: they’re immutable and lazy.

In [3]:
filePath="persons.json"
## For Linux "file:///Users/tirthalp/something/gs-spark-python/notebooks/11/persons.json"
## For Windows "C:\\Users\\tirthalp\\something\\gs-spark-python\\notebooks\\11\\persons.json"

df = spark.read.option("multiLine", "true") \
    .json(filePath)

type(df)

pyspark.sql.dataframe.DataFrame

##### Getting schema information

In [4]:
# Print the schema in a tree format
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- about: string (nullable = true)
 |-- address: string (nullable = true)
 |-- age: long (nullable = true)
 |-- balance: string (nullable = true)
 |-- company: string (nullable = true)
 |-- email: string (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- favoriteFruit: string (nullable = true)
 |-- friends: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- greeting: string (nullable = true)
 |-- guid: string (nullable = true)
 |-- index: long (nullable = true)
 |-- isActive: boolean (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- picture: string (nullable = true)
 |-- registered: string (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element

In [5]:
df.columns

['_id',
 'about',
 'address',
 'age',
 'balance',
 'company',
 'email',
 'eyeColor',
 'favoriteFruit',
 'friends',
 'gender',
 'greeting',
 'guid',
 'index',
 'isActive',
 'latitude',
 'longitude',
 'name',
 'phone',
 'picture',
 'registered',
 'tags']

In [6]:
df.dtypes

[('_id', 'string'),
 ('about', 'string'),
 ('address', 'string'),
 ('age', 'bigint'),
 ('balance', 'string'),
 ('company', 'string'),
 ('email', 'string'),
 ('eyeColor', 'string'),
 ('favoriteFruit', 'string'),
 ('friends', 'array<struct<id:bigint,name:string>>'),
 ('gender', 'string'),
 ('greeting', 'string'),
 ('guid', 'string'),
 ('index', 'bigint'),
 ('isActive', 'boolean'),
 ('latitude', 'double'),
 ('longitude', 'double'),
 ('name', 'string'),
 ('phone', 'string'),
 ('picture', 'string'),
 ('registered', 'string'),
 ('tags', 'array<string>')]

##### Accessing data

In [7]:
# Displays the content of the DataFrame to stdout in a tabular form

# By default, truncates long strings (i.e. strings more than 20 characters will be truncated and all cells will be aligned right)
df.show(2) # Display two records

# False = display complete strings without truncating
# df.show(2, False)

+--------------------+--------------------+--------------------+---+---------+--------+--------------------+--------+-------------+--------------------+------+--------------------+--------------------+-----+--------+----------+----------+----------+-----------------+--------------------+--------------------+--------------------+
|                 _id|               about|             address|age|  balance| company|               email|eyeColor|favoriteFruit|             friends|gender|            greeting|                guid|index|isActive|  latitude| longitude|      name|            phone|             picture|          registered|                tags|
+--------------------+--------------------+--------------------+---+---------+--------+--------------------+--------+-------------+--------------------+------+--------------------+--------------------+-----+--------+----------+----------+----------+-----------------+--------------------+--------------------+--------------------+
|5c6158

In [8]:
# Select only the "_id" and "name" columns

df.select("_id", "name").show(3, False)

+------------------------+-------------+
|_id                     |name         |
+------------------------+-------------+
|5c6158a17fd27b5a496f1383|Pope Soto    |
|5c6158a1c3a640b523a977ee|Lily Olson   |
|5c6158a1cd7259ec33888730|Elisa Burgess|
+------------------------+-------------+
only showing top 3 rows



In [9]:
# Select everybody, but increment the age by 5

df.select(df['name'], df['age'], df['age'] + 5).show()

+--------------+---+---------+
|          name|age|(age + 5)|
+--------------+---+---------+
|     Pope Soto| 39|       44|
|    Lily Olson| 23|       28|
| Elisa Burgess| 31|       36|
| Beasley Bryan| 37|       42|
|    Patti Barr| 23|       28|
|Betsy Callahan| 35|       40|
+--------------+---+---------+



In [10]:
# Filter data - Select people older than 35

df.filter(df['age'] > 35).show()

+--------------------+--------------------+--------------------+---+---------+--------+--------------------+--------+-------------+--------------------+------+--------------------+--------------------+-----+--------+----------+---------+-------------+-----------------+--------------------+--------------------+--------------------+
|                 _id|               about|             address|age|  balance| company|               email|eyeColor|favoriteFruit|             friends|gender|            greeting|                guid|index|isActive|  latitude|longitude|         name|            phone|             picture|          registered|                tags|
+--------------------+--------------------+--------------------+---+---------+--------+--------------------+--------+-------------+--------------------+------+--------------------+--------------------+-----+--------+----------+---------+-------------+-----------------+--------------------+--------------------+--------------------+
|

In [11]:
# Count people by age

df.groupBy("age").count().show()

+---+-----+
|age|count|
+---+-----+
| 31|    1|
| 39|    1|
| 37|    1|
| 35|    1|
| 23|    2|
+---+-----+



In [12]:
# Remove given column

df.drop("about").show(1)

+--------------------+--------------------+---+---------+--------+--------------------+--------+-------------+--------------------+------+--------------------+--------------------+-----+--------+----------+---------+---------+-----------------+--------------------+--------------------+--------------------+
|                 _id|             address|age|  balance| company|               email|eyeColor|favoriteFruit|             friends|gender|            greeting|                guid|index|isActive|  latitude|longitude|     name|            phone|             picture|          registered|                tags|
+--------------------+--------------------+---+---------+--------+--------------------+--------+-------------+--------------------+------+--------------------+--------------------+-----+--------+----------+---------+---------+-----------------+--------------------+--------------------+--------------------+
|5c6158a17fd27b5a4...|652 McDonald Aven...| 39|$2,868.75|BUZZNESS|popesoto@b

In [13]:
# Sorting

df.select("_id", "name", "age").orderBy("age").show()

+--------------------+--------------+---+
|                 _id|          name|age|
+--------------------+--------------+---+
|5c6158a1d8e19480b...|    Patti Barr| 23|
|5c6158a1c3a640b52...|    Lily Olson| 23|
|5c6158a1cd7259ec3...| Elisa Burgess| 31|
|5c6158a10c3e6ca29...|Betsy Callahan| 35|
|5c6158a15e2a20452...| Beasley Bryan| 37|
|5c6158a17fd27b5a4...|     Pope Soto| 39|
+--------------------+--------------+---+



##### Performing joins

When having related data in two DataFrames and want the resulting DataFrame to contain rows from both DataFrames with values common to both, then join can be performed.

Supported join types = inner, outer, left_outer, right_outer, or leftsemi.

For example,

val innerDataframe = someDf1.join(someDf2, someDf1("id") === 'uid)

The above performs an inner join. The outer join can be can performed by adding another argument.

val outerDataframe = someDf1.join(someDf2, someDf1("id") === 'uid, "outer")

##### Working with missing values

In [14]:
# Choose to drop the rows containing null or NaN (the Scala constant meaning “not a number”) values

cleanDf = df.na.drop()
cleanDf.count()

6

In [15]:
# mapping column names to replacement values,e.g. replace null values in the age column with zeroes

# Use -- df.na.fill(...)

## Converting DataFrames to RDDs

Every DataFrame has the lazily evaluated rdd field for accessing the underlying RDD. 

The resulting RDD contains elements of type org.apache.spark.sql.Row. Row has various get* functions for accessing column values
by column indexes (getString(index), getInt(index), getMap(index), and so forth. It also has a useful function for converting rows to strings (mimicking similar function available for Scala sequences): mkString(delimiter).

In [24]:
rdd = df.rdd
rdd.take(2)

[Row(_id='5c6158a17fd27b5a496f1383', about='Reprehenderit aute ullamco occaecat cillum ex ex ex consectetur consequat. Ut cupidatat commodo anim incididunt aliquip qui ex. Deserunt sint laboris quis commodo ut reprehenderit nulla. Laborum nisi veniam anim proident laborum sint est ex id veniam nostrud. Deserunt id enim sit ipsum duis deserunt irure. Sint eiusmod elit aliquip ex aliqua pariatur.\r\n', address='652 McDonald Avenue, Sunwest, Montana, 4757', age=39, balance='$2,868.75', company='BUZZNESS', email='popesoto@buzzness.com', eyeColor='blue', favoriteFruit='apple', friends=[Row(id=0, name='Norman Osborne'), Row(id=1, name='Bauer Martinez'), Row(id=2, name='Zimmerman Lancaster')], gender='male', greeting='Hello, Pope Soto! You have 3 unread messages.', guid='aa247d9e-6562-4e42-8f36-e95e98ea9e84', index=0, isActive=True, latitude=-56.739659, longitude=57.73213, name='Pope Soto', phone='+1 (911) 439-2209', picture='http://placehold.it/32x32', registered='2017-04-26T11:55:39 -06:-30

## Running SQL Queries Programmatically

* Spark SQL enables quering of DataFrames as database tables - The DataFrame DSL functionalities are accessible through SQL commands as an alternative interface for programming Spark SQL. When you write SQL commands in Spark SQL, they get translated into operations on DataFrames. 

* Temporary per-session and global tables.

* __The Catalyst optimizer__ of Spark makes SQL queries faster, which is the optimization engine that powers Spark SQL (as well as DataFrame API) since 2015. For more detail, [see](https://databricks.com/blog/2015/04/13/deep-dive-into-spark-sqls-catalyst-optimizer.html).

* SQL statements can be run by using the sql methods provided by spark.

* Tables schema can be inferred or explicitly specified

* Advanced window operations are also supported

* When using a SparkSession with Hive support, which is the recommended Spark SQL engine, the majority of Hive commands and data types are supported. Spark supports two SQL dialects: Spark’s SQL dialect and Hive Query Language (HQL). The Spark community recommends HQL (Spark 1.5) because HQL has a richer set of functionalities.

* Spark also lets you run SQL commands remotely, through a JDBC (and ODBC) server called __Thrift__. For more detail, [see](https://spark.apache.org/docs/latest/sql-distributed-sql-engine.html).

* In addition to the Spark shell, Spark also offers an SQL shell in the form of the __spark-sql__ CLI, which supports the same arguments as the spark-shell and spark-submit commands.

In [37]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("persons")

sqlDF = spark.sql("SELECT * FROM persons")
sqlDF.show(3)

+--------------------+--------------------+--------------------+---+---------+--------+--------------------+--------+-------------+--------------------+------+--------------------+--------------------+-----+--------+----------+----------+-------------+-----------------+--------------------+--------------------+--------------------+
|                 _id|               about|             address|age|  balance| company|               email|eyeColor|favoriteFruit|             friends|gender|            greeting|                guid|index|isActive|  latitude| longitude|         name|            phone|             picture|          registered|                tags|
+--------------------+--------------------+--------------------+---+---------+--------+--------------------+--------+-------------+--------------------+------+--------------------+--------------------+-----+--------+----------+----------+-------------+-----------------+--------------------+--------------------+--------------------

In [68]:
# Find male persons

spark.sql("SELECT _id, name, age FROM persons WHERE gender = 'male'").show()

+--------------------+-------------+---+
|                 _id|         name|age|
+--------------------+-------------+---+
|5c6158a17fd27b5a4...|    Pope Soto| 39|
|5c6158a15e2a20452...|Beasley Bryan| 37|
+--------------------+-------------+---+



## Using SQL functions to perform calculations on data

SQL functions fit into four categories:
* __Scalar functions__ return a single value for each row based on calculations on one or more columns.
* __Aggregate functions__ return a single value for a group of rows.
* __Window functions__ return several values for a group of rows.
* __User-defined functions__ include custom scalar or aggregate functions.

See [Spark SQL, Built-in functions](https://spark.apache.org/docs/latest/api/sql/index.html)

### Built-in Scalar functions

The built-in scalar functions = Math calculations, String operations and Date-time operations.

In [69]:
# Find average age of all males?

spark.sql("SELECT _id, name, UPPER(name) as name_ucase FROM persons").show()

+--------------------+--------------+--------------+
|                 _id|          name|    name_ucase|
+--------------------+--------------+--------------+
|5c6158a17fd27b5a4...|     Pope Soto|     POPE SOTO|
|5c6158a1c3a640b52...|    Lily Olson|    LILY OLSON|
|5c6158a1cd7259ec3...| Elisa Burgess| ELISA BURGESS|
|5c6158a15e2a20452...| Beasley Bryan| BEASLEY BRYAN|
|5c6158a1d8e19480b...|    Patti Barr|    PATTI BARR|
|5c6158a10c3e6ca29...|Betsy Callahan|BETSY CALLAHAN|
+--------------------+--------------+--------------+



### Built-in Aggregation functions

The [built-in DataFrames functions](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$) provide common aggregations such as count(), countDistinct(), avg(), max(), min(), etc. While those functions are designed for DataFrames, Spark SQL also has type-safe versions for some of them in Scala and Java to work with strongly typed Datasets. 

In [70]:
# Find average age of all males?

spark.sql("SELECT AVG(age) as avg_male_age FROM persons WHERE gender = 'male'").show()

+------------+
|avg_male_age|
+------------+
|        38.0|
+------------+



In [71]:
# Find the youngest female age?

spark.sql("SELECT MIN(age) AS min_female_age FROM persons WHERE gender = 'female'").show()

+--------------+
|min_female_age|
+--------------+
|            23|
+--------------+



### Built-in Window functions

Window functions = which operate over a range of rows in a DataFrame

Before 1.4, there were two kinds of functions supported by Spark SQL that could be used to calculate a single return value. Built-in functions or UDFs, such as substr or round, take values from a single row as input, and they generate a single return value for every input row. Aggregate functions, such as SUM or MAX, operate on a group of rows and calculate a single return value for every group.

While these are both very useful in practice, there is still a wide range of operations that cannot be expressed using these types of functions alone. Specifically, there was no way to both operate on a group of rows while still returning a single value for every input row. This limitation makes it hard to conduct various data processing tasks like calculating a moving average, calculating a cumulative sum, or accessing the values of a row appearing before the current row. Fortunately for users of Spark SQL, window functions fill this gap.

Window functions can be used to answer questions like: "Find the top selling mobile phone in last week's sell by performing operations". For this key considerations would be: Partition by = home product categories; Order by = no sold; Frame = one week; 

Ranking and analytic functions (e.g. first, last, lag, lead, ntile, cumeDist, rank, denseRank, percentRank, RowNumber) can be used as window functions - [See](https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html).

In [72]:
# Find top two oldest persons in each gender category (e.g. male and female)?

# For this - need to rank person in a category based on their age, and to pick the person based the ranking. 
# Below is the SQL query used to answer this question by using window function dense_rank

spark.sql("SELECT name, gender, age, rank \
FROM ( \
  SELECT name, gender, age, DENSE_RANK() OVER (PARTITION BY gender ORDER BY age DESC) as rank \
  FROM persons) tmp \
WHERE \
  rank <= 2").show()

+--------------+------+---+----+
|          name|gender|age|rank|
+--------------+------+---+----+
|Betsy Callahan|female| 35|   1|
| Elisa Burgess|female| 31|   2|
|     Pope Soto|  male| 39|   1|
| Beasley Bryan|  male| 37|   2|
+--------------+------+---+----+



In [73]:
# Find the difference between the age of each person and the most-oldest person in the respective gender category (male/female)?

import sys
from pyspark.sql.window import Window
import pyspark.sql.functions as func

windowSpec = Window \
    .partitionBy(df['gender']) \
    .orderBy(df['age'].desc()) \
    .rangeBetween(-sys.maxsize, sys.maxsize)

dataFrame = sqlContext.table("persons")

age_difference = (func.max(dataFrame['age']).over(windowSpec) - dataFrame['age'])

dataFrame.select(dataFrame['name'], dataFrame['gender'], dataFrame['age'], age_difference.alias("age_difference")).show()

+--------------+------+---+--------------+
|          name|gender|age|age_difference|
+--------------+------+---+--------------+
|Betsy Callahan|female| 35|             0|
| Elisa Burgess|female| 31|             4|
|    Lily Olson|female| 23|            12|
|    Patti Barr|female| 23|            12|
|     Pope Soto|  male| 39|             0|
| Beasley Bryan|  male| 37|             2|
+--------------+------+---+--------------+



### User-defined functions

In many situations, Spark SQL may not provide the specific functionality you need in a particular moment. UDFs let you extend the built-in functionalities of Spark SQL.

In short, Spark users are not limited to use the predefined or built-in functions only, rather can create their own user-defined functions too.

[See](https://docs.databricks.com/spark/latest/spark-sql/udf-python.html)

In [74]:
# How to define custom UDF in SQL, for example to count tags?

# Register a function as a UDF
def count_tags(tags):
    return len(tags)

spark.udf.register("UDF_COUNT_TAGS", count_tags)

# Call the UDF in Spark SQL
spark.sql("SELECT name, tags, UDF_COUNT_TAGS(tags) AS tags_count FROM persons ORDER BY tags_count").show()

+--------------+--------------------+----------+
|          name|                tags|tags_count|
+--------------+--------------------+----------+
|    Patti Barr|[minim, duis, ali...|         4|
|    Lily Olson|[exercitation, ad...|         6|
|     Pope Soto|[consectetur, lab...|         7|
| Elisa Burgess|[exercitation, cu...|         7|
|Betsy Callahan|[cupidatat, conse...|         7|
| Beasley Bryan|[elit, irure, ea,...|         7|
+--------------+--------------------+----------+



## Table catalog and Hive metastore

When executing SQL queries using Spark SQL, you can reference a DataFrame by its name by registering the DataFrame as a table. When you do that, Spark stores the table definition in the table catalog.

Hive functionalities can be enabled in Spark by calling enableHiveSupport() on a Builder object while constructing a SparkSession.

    val spark = SparkSession.builder().enableHiveSupport().getOrCreate()

__For Spark without Hive support__, a table catalog is implemented as a simple in-memory map, which means that table information lives in the driver’s memory and disappears with the Spark session. 

__SparkSession with Hive support__, on the other hand, uses a Hive metastore for implementing the table catalog. A Hive metastore is a persistent database, so DataFrame definitions remain available even if you close the Spark session and start a new one.

__Registering tables temporarily__: Hive support still enables you to create temporary table definitions. In both cases (Spark with or without Hive support), the createOrReplaceTempView method registers a temporary table.

__Registering tables permanetly__: Only SparkSession with Hive support can be used to register table definitions that will survive your application’s restarts (in other words, they’re persistent). By default, HiveContext creates a Derby database in the local working directory under the metastore_db subdirectory (or it reuses the database if it already exists). If you wish to change where the working directory is located, set the "hive.metastore.warehouse.dir" property in your "hive-site.xml" fileTo register a DataFrame as a permanent table, you need to use its write member.

__Working with the Spark table catalog__: Since version 2.0, Spark provides a facility for managing the table catalog. To see which tables are currently registered, use: 

    spark.catalog.listTables().show()

## Global Temporary View

Register the table as Global Temporary View to make it accessible across all Spark sessions.

__Temporary views__ in Spark SQL are session-scoped and will disappear if the session that creates it terminates. If you want to have a temporary view that is shared among all sessions and keep alive until the Spark application terminates, you can create a __global temporary view__. 

Global temporary view is tied to a system preserved database __global_temp__, and we must use the qualified name to refer it, e.g. SELECT * FROM global_temp.view1.

In [75]:
# Register the DataFrame as a global temporary view
df.createGlobalTempView("persons")

In [76]:
# Global temporary view is tied to a system preserved database `global_temp`
spark.sql("SELECT _id, name, friends.name as friends FROM global_temp.persons").show()

+--------------------+--------------+--------------------+
|                 _id|          name|             friends|
+--------------------+--------------+--------------------+
|5c6158a17fd27b5a4...|     Pope Soto|[Norman Osborne, ...|
|5c6158a1c3a640b52...|    Lily Olson|[Dillon Richard, ...|
|5c6158a1cd7259ec3...| Elisa Burgess|[Skinner Campbell...|
|5c6158a15e2a20452...| Beasley Bryan|[Jenkins Reyes, C...|
|5c6158a1d8e19480b...|    Patti Barr|[Aline Griffin, L...|
|5c6158a10c3e6ca29...|Betsy Callahan|[Langley Valdez, ...|
+--------------------+--------------+--------------------+



In [77]:
# Global temporary view is cross-session
spark.newSession().sql("SELECT _id, name, friends.name as friends FROM global_temp.persons").show()

+--------------------+--------------+--------------------+
|                 _id|          name|             friends|
+--------------------+--------------+--------------------+
|5c6158a17fd27b5a4...|     Pope Soto|[Norman Osborne, ...|
|5c6158a1c3a640b52...|    Lily Olson|[Dillon Richard, ...|
|5c6158a1cd7259ec3...| Elisa Burgess|[Skinner Campbell...|
|5c6158a15e2a20452...| Beasley Bryan|[Jenkins Reyes, C...|
|5c6158a1d8e19480b...|    Patti Barr|[Aline Griffin, L...|
|5c6158a10c3e6ca29...|Betsy Callahan|[Langley Valdez, ...|
+--------------------+--------------+--------------------+



## Saving and loading DataFrame data

* Spark has built-in support for several file formats and databases (generally called data sources in Spark). These include JDBC and Hive, and the JSON, ORC, and Parquet file formats. 
* For relational databases, Spark specifically supports dialects (meaning data-type mappings) for the MySQL and PostgreSQL databases.
* Data sources are pluggable, so you can add your own implementations. And you can download and use some external data, such as the [CSV](https://github.com/databricks/spark-csv), [Avro](https://github.com/databricks/spark-avro), and [Amazon Redshift](https://github.com/databricks/spark-redshift) data sources.
* Spark uses the metastore, which we covered in the previous section, to save information about where and how the data is stored. It uses data sources for saving and loading the actual data.

__Built-in data sources__

* __JSON__: Commonly used for web development and is popular as a lightweight alternative to XML. Simple, easy to use, and human-readable. However, not an efficient permanent data-storage format.
* __ORC__: Optimized row columnar (ORC) file format. Efficient way to store Hive data. Data from a single column is physically stored in close proximity.
* __Parquet__: Started outside Hive and was later integrated with it. Designed to be independent of any specific framework and free of unnecessary dependencies. More popular in the Hadoop ecosystem than the file format ORC. Columnar file format and also uses compression. Parquet is the default data source in Spark.

__Saving data__
    
    df.write.format("orc").mode("overwrite").option(...).saveAsTable("persons")

* DataFrame’s data is saved using the DataFrameWriter object, available as DataFrame’s write field. 
* In addition to the saveAsTable method, data can be saved using the save and insertInto methods. saveAsTable and insertInto save data into Hive tables and use the metastore in the process; save doesn’t. In case you’re not using Spark session with Hive support, the saveAsTable and insertInto methods create (or insert into) temporary tables. 
* You can configure all three methods with DataFrameWriter’s configuration functions.
    - "format": Specifies the file format for saving data (the data source name), which can be one of the built-in data sources (json, parquet, orc) or a named custom data source. When no format is specified, the default is parquet.
    - "mode": Specifies the save mode when a table or a file already exists. Possible values are overwrite (overwrites the existing data), append (appends the data), ignore (does nothing), and error (throws an exception); the default is error.
    - "option and options": Adds a single parameter name and a value (or a parametervalue map) to the data source configuration.
    - "partitionBy"" Specifies partitioning columns.

### Save in json format

In [78]:
spark.newSession().sql("SELECT _id, name, friends.name as friends FROM global_temp.persons") \
                  .write.format("json") \
                  .save("out\\persons-friends.json")

__About usage of coalesce()__

* In above code, coalesce() was not used. So the number of files written out would be equal to the number of partitions in the DataFrame.

* If coalesce() is used, then Spark would repartition the DataFrame into a single partition to write out a single file. For example,
    ```
    spark.newSession().sql("SELECT _id, name, friends.name as friends FROM global_temp.persons") \
                      .coalesce(1) \
                      .write.format("json") \
                      .save("out\\persons-friends.json")
    ```

### Save in parquet and load it again

In [79]:
# Save the result in parquet format

spark.newSession().sql("SELECT _id, name, friends.name as friends FROM global_temp.persons") \
    .write.format("parquet") \
    .save("out\\persons-friends.parquet")

In [80]:
# Read the data from parquet format

spark.read.parquet("out\\persons-friends.parquet").show()

+--------------------+--------------+--------------------+
|                 _id|          name|             friends|
+--------------------+--------------+--------------------+
|5c6158a17fd27b5a4...|     Pope Soto|[Norman Osborne, ...|
|5c6158a1c3a640b52...|    Lily Olson|[Dillon Richard, ...|
|5c6158a1cd7259ec3...| Elisa Burgess|[Skinner Campbell...|
|5c6158a15e2a20452...| Beasley Bryan|[Jenkins Reyes, C...|
|5c6158a1d8e19480b...|    Patti Barr|[Aline Griffin, L...|
|5c6158a10c3e6ca29...|Betsy Callahan|[Langley Valdez, ...|
+--------------------+--------------+--------------------+



### Save to RDBMS and load it using JDBC

Save the contents of DataFrame a with DataFrameWriter’s jdbc method. For example,

    val props = new java.util.Properties()
    props.setProperty("user", "user")
    props.setProperty("password", "password")
    postsDf.write.jdbc("jdbc:postgresql://postgresrv/mydb", "persons", props)
    
Load the data from RDBMS using DataFrameReader’s jdbc function. For example,

    val result = spark.read.jdbc("jdbc:postgresql://postgresrv/mydb", "persons", Array("viewCount > 3"), props)

## Summary

* DataFrames translate SQL code and DSL expressions into optimized, low-level RDD operations.
* DataFrames have become one of the most important features in Spark and have made Spark SQL the most actively developed Spark component.
* Three ways of creating DataFrames exist: by converting existing RDDs, by running SQL queries, or by loading external data.
* You can use DataFrame DSL operations to select, filter, group, and join data.
* DataFrames support scalar, aggregate, window, and user-defined functions.
* With the DataFrameNaFunctions class, accessible through DataFrame’s na field, you can deal with missing values in the dataset.
* SparkSQL has its own configuration method.
* Tables can be registered temporarily and permanently in the Hive metastore, which can reside in a local Derby database or in a remote relational database.
* The Spark SQL shell can be used to directly write queries referencing tables registered in the Hive metastore.
* Spark includes a Thrift server that clients can connect to over JDBC and ODBC and use to perform SQL queries.
* Data is loaded into DataFrames through DataFrameReader, available through SparkSession’s read field.
* Data is saved from DataFrames through DataFrameWriter, available through DataFrame’s write field.
* Spark’s built-in data sources are JSON, ORC, Parquet, and JDBC. Third-party data sources are available for download.
* Catalyst optimizer (the brain behind DataFrames) can optimize logical plans and create physical execution plans.
* The Tungsten project introduced numerous performance improvements through binary, cache-friendly encoding of objects, on-heap and off-heap allocation, and a new shuffle manager.
* DataSets are an experimental feature similar to DataFrames, but they enable you to store plain Java objects instead of generic Row containers.
* See important [classes of Spark SQL and DataFrames](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html).